# Import libraries

In [2]:
import os
import sys
import pypots
import numpy as np
import benchpots
import matplotlib.pyplot as plt
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
from pypots.utils.random import set_random_seed
from functions.toolkits import toolkits
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from MAEModify.error import calc_mae

2025-05-07 20:49:23.223971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746661763.239140  104699 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746661763.243924  104699 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 20:49:23.260274: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidge


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Load Dataset

In [3]:
set_random_seed()
physionet2012_dataset = benchpots.datasets.preprocess_physionet2012(subset="all", rate=0.1)
print(physionet2012_dataset.keys())

2025-05-07 20:49:57 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-05-07 20:49:57 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-05-07 20:49:57 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-05-07 20:49:57 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-05-07 20:49:57 [INFO]: Loaded successfully!
2025-05-07 20:50:10 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-07 20:50:10 [INFO]: 68807 values masked out in the val set as ground truth, take 9.97% of the original observed values
2025-05-07 20:50:10 [INFO]: 86319 values masked out in the test set as ground truth, take 9.99% of the original observed v

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])


In [4]:
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}

dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}

dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
}

test_X_indicating_mask = np.isnan(physionet2012_dataset['test_X_ori']) ^ np.isnan(physionet2012_dataset['test_X'])
test_X_ori = np.nan_to_num(physionet2012_dataset['test_X_ori']) 

# Train/Load Models

## SAITS

In [5]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  
    MIT_weight=1,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-07 20:50:26 [INFO]: No given device, using default device: cpu
2025-05-07 20:50:26 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-07 20:50:26 [INFO]: Using customized MAE as the training loss function.
2025-05-07 20:50:26 [INFO]: Using customized MSE as the validation metric function.
2025-05-07 20:50:26 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


In [6]:
saits.load("../mae/tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots")

2025-05-07 20:50:29 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots


## BRITS

In [7]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-07 20:50:31 [INFO]: No given device, using default device: cpu
2025-05-07 20:50:31 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-07 20:50:31 [INFO]: Using customized MAE as the training loss function.
2025-05-07 20:50:31 [INFO]: Using customized MSE as the validation metric function.
2025-05-07 20:50:31 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


In [8]:
brits.load("../mae/tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots")

2025-05-07 20:50:45 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots


## US-GAN

In [9]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    epochs=10,
    patience=3,
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-07 20:52:07 [INFO]: No given device, using default device: cpu
2025-05-07 20:52:07 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-07 20:52:07 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


In [10]:
us_gan.load("../mae/tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots")

2025-05-07 20:52:09 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots


## GP-VAE

In [11]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-07 20:52:41 [INFO]: No given device, using default device: cpu
2025-05-07 20:52:41 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-07 20:52:41 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


In [12]:
gp_vae.load("../mae/tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots")

2025-05-07 20:52:55 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots


## MRNN

In [13]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-07 20:54:18 [INFO]: No given device, using default device: cpu
2025-05-07 20:54:18 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-07 20:54:18 [INFO]: Using customized RMSE as the training loss function.
2025-05-07 20:54:18 [INFO]: Using customized MSE as the validation metric function.
2025-05-07 20:54:18 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


In [14]:
mrnn.load("../mae/tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots")

2025-05-07 20:54:22 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots


# Imputation Models

## SAITS

In [15]:
saits_results = saits.predict(dataset_for_testing)
saits_imputation = saits_results["imputation"]

## BRITS

In [16]:
brits_results = brits.predict(dataset_for_testing)
brits_imputation = brits_results["imputation"]

## USGAN

In [17]:
us_gan_results = us_gan.predict(dataset_for_testing)
us_gan_imputation = us_gan_results["imputation"]

## GPVAE

In [18]:
gp_vae_results = gp_vae.predict(dataset_for_testing)
gp_vae_imputation = gp_vae_results["imputation"]

## MRNN

In [19]:
mrnn_results = mrnn.predict(dataset_for_testing)
mrnn_imputation = mrnn_results["imputation"]

# AE/MAE Models

## SAITS

In [20]:
saits_mae, saits_ae = calc_mae(
    saits_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## BRITS

In [21]:
brits_mae, brits_ae = calc_mae(
    brits_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## USGAN

In [22]:
usgan_mae, usgan_ae = calc_mae(
    us_gan_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## GPVAE

In [23]:
gp_vae_imputation = np.squeeze(gp_vae_imputation, axis=1)

gpvae_mae, gpvae_ae = calc_mae(
    gp_vae_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## MRNN

In [24]:
mrnn_mae, mrnn_ae = calc_mae(
    mrnn_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

# Gini Confidence Interval

## SAITS

In [8]:
saits_ae = saits_ae.reshape(len(saits_ae) * 48 * 37)
saits_gini = toolkits.bootstrap(saits_ae, 9000)

In [9]:
lower_bound_saits = np.percentile(saits_gini, 2.5)
upper_bound_saits = np.percentile(saits_gini, 97.5)

print(lower_bound_saits)
print(upper_bound_saits)

0.5483582258241262
0.5490869267782328


In [10]:
mean_bound_saits = (lower_bound_saits+upper_bound_saits)/2
print(mean_bound_saits)

0.5487225763011795


## BRITS

In [ ]:
brits_ae = brits_ae.reshape(len(brits_ae) * 48 * 37)
brits_gini = toolkits.bootstrap(brits_ae, 9000)

In [ ]:
lower_bound_brits = np.percentile(brits_gini, 2.5)
upper_bound_brits = np.percentile(brits_gini, 97.5)

print(lower_bound_brits)
print(upper_bound_brits)

In [ ]:
mean_bound_brits = (lower_bound_brits+upper_bound_brits)/2
print(mean_bound_brits)

## USGAN

In [ ]:
usgan_ae = usgan_ae.reshape(len(usgan_ae) * 48 * 37)
usgan_gini = toolkits.bootstrap(usgan_ae, 9000)

In [ ]:
lower_bound_usgan = np.percentile(usgan_gini, 2.5)
upper_bound_usgan = np.percentile(usgan_gini, 97.5)

print(lower_bound_usgan)
print(upper_bound_usgan)

In [ ]:
mean_bound_usgan = (lower_bound_usgan+upper_bound_usgan)/2
print(mean_bound_usgan)

## GPVAE

In [ ]:
gpvae_ae = gpvae_ae.reshape(len(gpvae_ae) * 48 * 37)
gpvae_gini = toolkits.bootstrap(gpvae_ae, 9000)

In [ ]:
lower_bound_gpvae = np.percentile(gpvae_gini, 2.5)
upper_bound_gpvae = np.percentile(gpvae_gini, 97.5)

print(lower_bound_gpvae)
print(upper_bound_gpvae)

In [ ]:
mean_bound_gpvae = (lower_bound_gpvae+upper_bound_gpvae)/2
print(mean_bound_gpvae)

## MRNN

In [ ]:
mrnn_ae = mrnn_ae.reshape(len(mrnn_ae) * 48 * 37)
mrnn_gini = toolkits.bootstrap(mrnn_ae, 9000)

In [ ]:
lower_bound_mrnn = np.percentile(mrnn_gini, 2.5)
upper_bound_mrnn = np.percentile(mrnn_gini, 97.5)

print(lower_bound_mrnn)
print(upper_bound_mrnn)

In [ ]:
mean_bound_mrnn = (lower_bound_mrnn+upper_bound_mrnn)/2
print(mean_bound_mrnn)